In [28]:
!pip install pyspark

In [29]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [30]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
home_sales_df.show()



+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [31]:
# 2. Create a temporary view of the DataFrame.

home_sales_df.createOrReplaceTempView('home_sales')

In [32]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("""select Round(Avg(price),2) as average_price ,date_built  from home_sales where bedrooms=4 group by date_built order by date_built desc""").show()


+-------------+----------+
|average_price|date_built|
+-------------+----------+
|    296576.69|      2017|
|    296050.24|      2016|
|    307908.86|      2015|
|    299073.89|      2014|
|    299999.39|      2013|
|    298233.42|      2012|
|     302141.9|      2011|
|    296800.75|      2010|
+-------------+----------+



In [33]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""select Round(Avg(price),2) as average_price ,date_built  from home_sales where bedrooms=3 and bathrooms=3 group by date_built order by date_built desc""").show()


+-------------+----------+
|average_price|date_built|
+-------------+----------+
|    292676.79|      2017|
|    290555.07|      2016|
|     288770.3|      2015|
|    290852.27|      2014|
|    295962.27|      2013|
|    293683.19|      2012|
|    291117.47|      2011|
|    292859.62|      2010|
+-------------+----------+



In [34]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("""select Round(Avg(price),2) as average_price ,
date_built
from home_sales
where bedrooms=3 and bathrooms=3 and floors=2 and sqft_lot>=2000
group by date_built
order by date_built desc""").show()


+-------------+----------+
|average_price|date_built|
+-------------+----------+
|    282026.59|      2017|
|     287812.6|      2016|
|    291788.36|      2015|
|    294195.13|      2014|
|    295142.13|      2013|
|    295858.68|      2012|
|    281413.45|      2011|
|    280447.23|      2010|
+-------------+----------+



In [35]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""select view,
Round(Avg(price),2) as average_price
from home_sales
group by view
having average_price >= 350000
order by view desc""").show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

--- 1.129885196685791 seconds ---


In [36]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [37]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [38]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""select view,
Round(Avg(price),2) as average_price
from home_sales
group by view
having average_price >= 350000
order by view desc""").show()



print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|average_price|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

--- 0.8542535305023193 seconds ---


In [39]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_df.write.partitionBy("date_built").mode("overwrite").parquet("P_home_sales")

In [40]:
# 11. Read the formatted parquet data.
p_home_sales_df=spark.read.parquet('P_home_sales')

In [41]:
# 12. Create a temporary table for the parquet data.
p_home_sales_df.createOrReplaceTempView('p_temp_home_sales')

In [42]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()


spark.sql("""select view,
Round(Avg(price),2) as average_price
from p_temp_home_sales
group by view
having average_price >= 350000
order by view desc""").show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

--- 1.5337347984313965 seconds ---


In [43]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [44]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")


False